In [1]:
import datetime as dt  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

sns.set()
%matplotlib inline

In [2]:
# Load the data
calls = pd.read_csv('../data/callsforservicefiltered.csv', header=0, low_memory=True, 
                    memory_map=True)
print(len(calls))
#calls.dtypes

1124451


C:\Users\ltran\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1. Clean Up

In [3]:
# Filter out rows where cancelled calls field = True, report only field = True.

calls = calls[calls.rptonly != True]
calls = calls[calls.cancelled != True]
print(len(calls))

1101654


In [4]:
# Filter out rows where nature = Transport to Animal Shelter.

calls = calls[calls.nature != 'TRANSPORT TO ANIMAL SHELTER']
print(len(calls))

1083514


In [5]:
# Filter out "Traffic Stop" because the response time is relevant since it does not get call from dispather
calls = calls[calls.nature != 'TRAFFIC STOP']
print(len(calls))

969028


In [6]:
# Filter out "911 UNKNOWN" calls since this could not be classified
calls = calls[calls.nature != '911 UNKNOWN']
print(len(calls))

965217


In [7]:
calls.isnull().sum()

calltime           0
callsource         3
city             502
nature             0
nature2       935611
priority         245
rptonly            0
service            0
agency             0
district       81525
statbeat       81523
ra             81528
gp             84358
primeunit        411
cancelled          0
closecode          0
timeclose          0
calldow            0
secs2rt            0
secs2di            0
secs2en            0
secs2ar            0
secs2tr            0
secs2lc            0
secsdi2en          0
secsdi2ar          0
secsar2tr          0
secsar2lc          0
secsrt2dsp         0
secstr2lc          0
firstdisp        642
dtype: int64

In [8]:
#Remove unnessessary columns
calls.drop(columns = ['statbeat','ra'], inplace=True)
calls

,calltime,callsource,city,nature,nature2,priority,rptonly,service,agency,district,...,secs2ar,secs2tr,secs2lc,secsdi2en,secsdi2ar,secsar2tr,secsar2lc,secsrt2dsp,secstr2lc,firstdisp
1,2012-07-01T00:09:07Z,E911,G,HEMORRHAGE / LACERATIONS,NaN,4,False,EMS,EMS,B6,...,571,2059,3025,48,393,1881,2454,43,965,M261
3,2012-07-01T00:15:56Z,E911,W,SEIZURES-CONVULSIONS,NaN,1,False,EMS,EMS,B5,...,915,1519,5437,45,854,1457,4522,30,3918,M2
4,2012-07-01T00:17:20Z,E911,W,SEIZURES-CONVULSIONS,NaN,P,False,FIRE,GCF,ALAM,...,685,0,1586,279,677,0,900,8,0,E44
5,2012-07-01T00:19:24Z,E911,G,FALLS-BACK INJURIES(TRAUMATIC),NaN,3,False,EMS,EMS,B4B,...,0,0,339,23,0,0,0,10,0,M241
6,2012-07-01T00:24:42Z,E911,G,SICK PERSON,NaN,4,False,EMS,EMS,B1,...,306,1335,2602,22,224,1253,2295,29,1267,M211
8,2012-07-01T00:30:03Z,W911,H,CHEST PAIN,NaN,1,False,EMS,EMS,B2,...,340,1107,2430,47,277,1043,2090,17,1323,M221
9,2012-07-01T00:33:11Z,PHONE,SU,DOMESTIC DISPUTE,NaN,1,False,LAW,GCSD,SD1,...,1148,0,2257,105,290,0,1109,102,0,S314
10,2012-07-01T00:37:41Z,SELF,G,LEGAL PAPER / COMPLAINT,NaN,2,False,LAW,GCSD,GPD1,...,0,0,3734,0,0,0,3734,0,0,S324
11,2012-07-01T00:46:35Z,E911,G,BREATHING PROBLEMS,NaN,1,False,EMS,EMS,B6,...,689,0,1200,26,604,0,511,56,0,MA32
12,2012-07-01T00:47:14Z,E911,G,SICK PERSON,NaN,4,False,EMS,EMS,B6,...,323,753,2339,21,210,640,2015,37,1585,MA32


In [9]:
calls['actualnature'] = [x['nature2'] if x['nature'] is np.nan and x['nature']!=x['nature2']
                        else x['nature'] for i,x in calls.iterrows()]
calls = calls.drop(columns = ['nature','nature2'])
calls = calls.rename(columns={'actualnature' : 'nature'})

In [10]:
calls['calltime'] = pd.to_datetime(calls.calltime)
calls['year'] = calls.calltime.apply(lambda x: x.date().year)
calls['month'] = calls.calltime.apply(lambda x: x.date().month)
calls['dayOfMonth'] = calls.calltime.apply(lambda x: x.date().day)
calls['hour'] = calls.calltime.apply(lambda x: x.time().hour)
calls = calls.rename(columns={'calldow' : 'dayOfWeek'})
calls = calls.drop(columns = ['timeclose', 'calltime'], axis = 1)
calls

,callsource,city,priority,rptonly,service,agency,district,gp,primeunit,cancelled,...,secsar2tr,secsar2lc,secsrt2dsp,secstr2lc,firstdisp,nature,year,month,dayOfMonth,hour
1,E911,G,4,False,EMS,EMS,B6,627,M261,False,...,1881,2454,43,965,M261,HEMORRHAGE / LACERATIONS,2012,7,1,0
3,E911,W,1,False,EMS,EMS,B5,513,M2,False,...,1457,4522,30,3918,M2,SEIZURES-CONVULSIONS,2012,7,1,0
4,E911,W,P,False,FIRE,GCF,ALAM,44C,E44,False,...,0,900,8,0,E44,SEIZURES-CONVULSIONS,2012,7,1,0
5,E911,G,3,False,EMS,EMS,B4B,406,M241,False,...,0,0,10,0,M241,FALLS-BACK INJURIES(TRAUMATIC),2012,7,1,0
6,E911,G,4,False,EMS,EMS,B1,119,M211,False,...,1253,2295,29,1267,M211,SICK PERSON,2012,7,1,0
8,W911,H,1,False,EMS,EMS,B2,202,M221,False,...,1043,2090,17,1323,M221,CHEST PAIN,2012,7,1,0
9,PHONE,SU,1,False,LAW,GCSD,SD1,SD12,S314,False,...,0,1109,102,0,S314,DOMESTIC DISPUTE,2012,7,1,0
10,SELF,G,2,False,LAW,GCSD,GPD1,P110,S324,False,...,0,3734,0,0,S324,LEGAL PAPER / COMPLAINT,2012,7,1,0
11,E911,G,1,False,EMS,EMS,B6,620,M231,False,...,0,511,56,0,MA32,BREATHING PROBLEMS,2012,7,1,0
12,E911,G,4,False,EMS,EMS,B6,602,MA32,False,...,640,2015,37,1585,MA32,SICK PERSON,2012,7,1,0


In [11]:
calls.isnull().sum()

callsource        3
city            502
priority        245
rptonly           0
service           0
agency            0
district      81525
gp            84358
primeunit       411
cancelled         0
closecode         0
dayOfWeek         0
secs2rt           0
secs2di           0
secs2en           0
secs2ar           0
secs2tr           0
secs2lc           0
secsdi2en         0
secsdi2ar         0
secsar2tr         0
secsar2lc         0
secsrt2dsp        0
secstr2lc         0
firstdisp       642
nature            0
year              0
month             0
dayOfMonth        0
hour              0
dtype: int64

In [ ]:
# split the data up by agency
ems_calls = calls[calls.agency == 'EMS']
aco_calls = calls[calls.agency == 'ACO']
gcsd_calls = calls[calls.agency == 'GCSD']
gcf_calls = calls[calls.agency == 'GCF']

In [ ]:
ems_calls

# Progress report 3

# Data Anayltics Nature Vs. Response time!
## Goals:
### - Basic statistics and clean data some more for our purposes
### - Check the Distribution of the of response time and see patterns
### - Check Relationships of response time and  number of calls
### - Check Relationships of response time and month of the years

###  1.1 EMS - Guilford county Emergency Medical Service

In [ ]:
ems_groupby_nature_size = ems_calls.groupby('nature')['secsdi2en'].size().sort_values(ascending=False).reset_index(name='second')
ems_groupby_nature_size
#ems_groupby_nature_size


In [ ]:
ems_groupby_nature_mean = ems_calls.groupby('nature')['secsdi2en'].mean().sort_values(ascending=False).reset_index(name='second_avg')
ems_groupby_nature_mean

In [ ]:
ems_merged = pd.merge(ems_groupby_nature_size,ems_groupby_nature_mean,on='nature')
ems_merged

In [ ]:
ems_merged["second"].describe()


### 1.2 EMS - Find Outlier

In [ ]:
# IQR
iqr_ems = ems_merged["second"].quantile(0.75) - ems_merged["second"].quantile(0.25)
iqr_ems



In [ ]:
# median
median_size_ems = ems_merged["second"].quantile(0.50)
median_size_ems


In [ ]:
# lower outlier bound
lower_outlier_bound_ems = ems_merged["second"].quantile(0.25) - 1.5 * iqr_ems
lower_outlier_bound_ems

In [ ]:
# upper outlier bound
upper_outlier_boud_ems = ems_merged["second"].quantile(0.75) +  1.5 * iqr_ems
upper_outlier_boud_ems


In [ ]:
# new data range after outlier has been removed
ems_outlier_removed = ems_merged[(ems_merged['second'] > lower_outlier_bound_ems) & (ems_merged['second'] < upper_outlier_boud_ems) ]
ems_outlier_removed


### 1.3 EMS - Graph

In [ ]:
print (ems_outlier_removed['second'].corr(ems_outlier_removed['second_avg']))

In [ ]:
ems_plot = ems_outlier_removed.plot(kind='bar', title = "Gcf_response time", figsize=(15,7))
ems_plot.set_xlabel("nature index")
ems_plot.set_ylabel("second")

In [ ]:
ems_merged.boxplot(column="second",
               return_type='axes',
               figsize=(10,10))

plt.text(x=0.74, y=ems_merged["second"].quantile(0.75), s="3rd Quartile")
plt.text(x=0.8, y=ems_merged["second"].quantile(0.50), s="Median")
plt.text(x=0.75, y=ems_merged["second"].quantile(0.25), s="1st Quartile")
plt.text(x=0.9, y=ems_merged["second"].quantile(0), s="Min")
plt.text(x=0.9, y=ems_merged["second"].quantile(1), s="Max")
plt.text(x=0.7, y=iqr_ems, s="IQR", rotation=90, size=10)

In [ ]:

#plt.xticks( site2['Cl'], site2.index.values ) # location, labels
#plt.plot( site2['Cl'] )
#plt.show()



ems_plot = ems_outlier_removed.plot(kind='bar', title = "Ems_response time", figsize=(15,7))
ems_plot.set_xlabel("nature index")
ems_plot.set_ylabel("second")


In [ ]:
ems_outlier_removed['second'].corr(ems_outlier_removed['second'])

### 2.1 ACO - Animal control

In [ ]:
aco_groupby_nature_size = aco_calls.groupby('nature')['secsdi2en'].size().sort_values(ascending=False).reset_index(name='second')
aco_groupby_nature_size

In [ ]:
aco_groupby_nature_mean = aco_calls.groupby('nature')['secsdi2en'].mean().sort_values(ascending=False).reset_index(name='second_avg')
aco_groupby_nature_mean

In [ ]:
# mergering 
aco_merged = pd.merge(aco_groupby_nature_size,aco_groupby_nature_mean,on='nature')
aco_merged

In [ ]:
aco_merged["second"].describe()

### 2.2 ACO - Find Outlier

In [ ]:
# IQR
iqr_aco = aco_merged["second"].quantile(0.75) - aco_merged["second"].quantile(0.25)
iqr_aco

In [ ]:
# median
median_size_aco = aco_merged["second"].quantile(0.50)
median_size_aco

In [ ]:
# upper outlier bound
upper_outlier_bound_aco = ems_merged["second"].quantile(0.75) +  1.5 * iqr_aco
upper_outlier_bound_aco

In [ ]:
# lower outlier bound
lower_outlier_bound_aco = aco_merged["second"].quantile(0.25) - 1.5 * iqr_aco
lower_outlier_bound_aco

In [ ]:
# new data range after outlier has been removed
aco_outlier_removed = aco_merged[(aco_merged['second'] > lower_outlier_bound_aco) & (aco_merged['second'] < upper_outlier_bound_aco) ]
aco_outlier_removed

### 2.3 ACO graph

In [ ]:
aco_merged.boxplot(column="second",
               return_type='axes',
               figsize=(10,10))

plt.text(x=0.74, y=aco_merged["second"].quantile(0.75), s="3rd Quartile")
plt.text(x=0.8, y=aco_merged["second"].quantile(0.50), s="Median")
plt.text(x=0.75, y=aco_merged["second"].quantile(0.25), s="1st Quartile")
plt.text(x=0.9, y=aco_merged["second"].quantile(0), s="Min")
plt.text(x=0.9, y=aco_merged["second"].quantile(1), s="Max")
plt.text(x=0.7, y=iqr_aco, s="IQR", rotation=90, size=10)

In [ ]:
aco_plot = aco_outlier_removed.plot(kind='bar', title = "ACO_response time", figsize=(15,7))
aco_plot.set_xlabel("nature index")
aco_plot.set_ylabel("second")

### 3.1 GCSD -Guilford County Sheriff Department

In [ ]:
gcsd_groupby_nature_size = gcsd_calls.groupby('nature')['secsdi2en'].size().sort_values(ascending=False).reset_index(name='second')
gcsd_groupby_nature_size

In [ ]:
gcsd_groupby_nature_mean = gcsd_calls.groupby('nature')['secsdi2en'].mean().sort_values(ascending=False).reset_index(name='second_avg')
gcsd_groupby_nature_mean

In [ ]:
# mergering 
gcsd_merged = pd.merge(aco_groupby_nature_size,aco_groupby_nature_mean,on='nature')
gcsd_merged

In [ ]:
gcsd_merged["second"].describe()

### 3.2 GCSD - outlier

In [ ]:
# IQR
iqr_gcsd = gcsd_merged["second"].quantile(0.75) - gcsd_merged["second"].quantile(0.25)
iqr_gcsd

In [ ]:
# median
median_size_gcsd = gcsd_merged["second"].quantile(0.50)
median_size_gcsd

In [ ]:
# lower outlier bound
lower_outlier_bound_gcsd = gcsd_merged["second"].quantile(0.25) - 1.5 * iqr_gcsd
lower_outlier_bound_gcsd

In [ ]:
# upper outlier bound
upper_outlier_boud_gcsd = gcsd_merged["second"].quantile(0.75) +  1.5 * iqr_gcsd
upper_outlier_boud_gcsd

In [ ]:
# new data range after outlier has been removed
gcsd_outlier_removed = gcsd_merged[(gcsd_merged['second'] > lower_outlier_bound_gcsd) & (gcsd_merged['second'] < upper_outlier_boud_gcsd) ]
gcsd_outlier_removed

### 3.3 GCSD - graph

In [ ]:
gcsd_merged.boxplot(column="second",
               return_type='axes',
               figsize=(10,10))

plt.text(x=0.74, y=gcsd_merged["second"].quantile(0.75), s="3rd Quartile")
plt.text(x=0.8, y=gcsd_merged["second"].quantile(0.50), s="Median")
plt.text(x=0.75, y=gcsd_merged["second"].quantile(0.25), s="1st Quartile")
plt.text(x=0.9, y=gcsd_merged["second"].quantile(0), s="Min")
plt.text(x=0.9, y=gcsd_merged["second"].quantile(1), s="Max")
plt.text(x=0.7, y=iqr_gcsd, s="IQR", rotation=90, size=10)

In [ ]:
gcsd_plot = gcsd_outlier_removed.plot(kind='bar', title = "gcsd_response time", figsize=(15,7))
gcsd_plot.set_xlabel("nature index")
gcsd_plot.set_ylabel("second")

###  4.1 GFD - Guilford County Fire Department

In [ ]:
gcf_groupby_nature_size = gcf_calls.groupby('nature')['secsdi2en'].size().sort_values(ascending=False).reset_index(name='second')
gcf_groupby_nature_size


In [ ]:
gcf_groupby_nature_mean = gcf_calls.groupby('nature')['secsdi2en'].mean().sort_values(ascending=False).reset_index(name='second_avg')
gcf_groupby_nature_mean

In [ ]:
# mergering 
gcf_merged = pd.merge(gcf_groupby_nature_size,gcf_groupby_nature_mean,on='nature')
gcf_merged

In [ ]:
gcf_merged["second"].describe()

### 4.2 GCF - outlier

In [ ]:
# IQR
iqr_gcf = gcf_merged["second"].quantile(0.75) - gcf_merged["second"].quantile(0.25)
iqr_gcf

In [ ]:
# median
median_size_gcf = gcf_merged["second"].quantile(0.50)
median_size_gcf

In [ ]:
# lower outlier bound
lower_outlier_bound_gcf = gcf_merged["second"].quantile(0.25) - 1.5 * iqr_gcf
lower_outlier_bound_gcf

In [ ]:
# upper outlier bound
upper_outlier_boud_gcf = gcf_merged["second"].quantile(0.75) +  1.5 * iqr_gcf
upper_outlier_boud_gcf

In [ ]:
# new data range after outlier has been removed
gcf_outlier_removed = gcf_merged[(gcf_merged['second'] > lower_outlier_bound_gcf) & (gcf_merged['second'] < upper_outlier_boud_gcf) ]
gcf_outlier_removed

### 4.3 GCF - graph

In [ ]:
gcf_merged.boxplot(column="second",
               return_type='axes',
               figsize=(10,10))

plt.text(x=0.74, y=gcf_merged["second"].quantile(0.75), s="3rd Quartile")
plt.text(x=0.8, y=gcf_merged["second"].quantile(0.50), s="Median")
plt.text(x=0.75, y=gcf_merged["second"].quantile(0.25), s="1st Quartile")
plt.text(x=0.9, y=gcf_merged["second"].quantile(0), s="Min")
plt.text(x=0.9, y=gcf_merged["second"].quantile(1), s="Max")
plt.text(x=0.7, y=iqr_gcf, s="IQR", rotation=90, size=10)

In [ ]:
gcf_plot = gcf_outlier_removed.plot(kind='bar', title = "Gcf_response time", figsize=(15,7))
gcf_plot.set_xlabel("nature index")
gcf_plot.set_ylabel("second")

Progress made towards the task.
Formulate the question.
Why was the question asked?
Describe the method
What was done to solve for the question?
Describe outcomes.
What were the outcomes
What are their implications
Tasks could be
Data analytics 



## Final Presention Preparation

### Keep the data split into 4 agencies
### Graph the correlation // heat map --> select the best feature
### Prepare the data for training

#### 1. Heatmap/corl on 